In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import scipy


def inite(elec,orb):
    config=[]
    list1=[]
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
                
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)

                        list1.append(config)
                        config=[]
    return list1

def gr_state(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

    wires=range(qubits)
    params = np.zeros(len(singles) + len(doubles))
    # Define the device
    dev = qml.device("default.qubit", wires=qubits, shots=shots)
    
    # Define the qnode
    if shots==None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                 s_wires=s_wires, d_wires=d_wires,
                                                 hf_state=hf_state)
        
    print(f'ground state energy is: {circuit(params, wires, s_wires, d_wires, hf_state)}')
    return params

def ex_state(symbols, geometry, electrons, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)

    null_state = np.zeros(qubits,int)
    list1 = inite(electrons,qubits)
    values =[]
    if shots==0:
        #dev = qml.device("default.qubit", wires=qubits)
        dev = qml.device("default.qubit", wires=qubits)
    else:
        #dev = qml.device("default.qubit", wires=qubits,shots=shots)
        dev = qml.device("default.qubit", wires=qubits,shots=shots)
    #circuit for diagonal part
    @qml.qnode(dev)
    def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
        for w in occ:
            qml.X(wires=w)
        qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
        return qml.expval(H)
    #circuit for off-diagonal part
    @qml.qnode(dev)
    def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
        for w in occ1:
            qml.X(wires=w)
        first=-1
        for v in occ2:
            if v not in occ1:
                if first==-1:
                    first=v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first,v])
        for v in occ1:
            if v not in occ2:
                if first==-1:
                    first=v
                    qml.Hadamard(wires=v)
                else:
                    qml.CNOT(wires=[first,v])
        qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
        return qml.expval(H)
    #final M matrix
    M = np.zeros((len(list1),len(list1)))
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i == j:
                M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
    for i in range(len(list1)):
        for j in range(len(list1)):
            if i!=j:
                Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
    eig,evec=np.linalg.eig(M)
    values.append(np.sort(eig))
    return values



# Define the molecule
#in bohr 
symbols  = [ 'H', 'H']
geometry = np.array([[0.0, 0.0,  0.0],
                    [0.0, 0.0,  2.83459]], requires_grad = False)
electrons = 2
charge = 0
params = gr_state(symbols,geometry,electrons,charge,max_iter=50)
#params_exact=params

ex_energy = ex_state(symbols, geometry, electrons, charge,params)
print('excited state energies:\n', ex_energy)

/Users/prince.kwao/miniconda3/envs/cwp/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:209: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


ground state energy is: -0.998149285713912
excited state energies:
 [tensor([-0.89058484, -0.43151291, -0.30719265], requires_grad=True)]
